In [ ]:
from service import config
config.initialize()

from service.MatchSys import MatchSys
ms = MatchSys(
        name='teat_sys',
        ltp_model_path=r'D:\Code\MyLongTimeProject\A\QQ-Bot-And-Tool\data\LtpModel',
        database_uri='sqlite:///data/db.sqlite3',
        text_vec_model_path=r'D:\Code\MyLongTimeProject\A\QQ-Bot-And-Tool\data\model.pkl',
        vector_similarity_rate=0.7,
        most_similar_number=10,
        vector_match_times=5,

    )

@ms.handle_function_declaration(commands = ['天气怎么样'])
def get_weather_abstract_by_hefeng(input_statement):
    import requests
    from bs4 import BeautifulSoup
    get_location_url = 'https://geoapi.qweather.com/v2/city/lookup'
    address = input_statement.text.split('天气怎么样')[0]
    get_loction_params = {
        'key': config.SERVICE_CONFIG.he_feng_key,
        'location': address
    }
    location_data = requests.get(get_location_url,get_loction_params).json()
    print(location_data)
    location_data = location_data['location'][0]
    page = requests.get(location_data['fxLink'])
    soup = BeautifulSoup(page.text, 'html.parser')
    abstract = soup.find_all('div', class_='current-abstract')[0]
    return abstract.text.strip() + '\n 详细信息：'+location_data['fxLink']

In [ ]:
import pandas as pd
map = {}
# data = pd.read_excel(r'C:\Users\Administrator\Documents\GitHub\QQ-Bot-And-Tool\data\FixedReply\傲娇系二次元bot词库5千词V1.2.xlsx',header=None, sheet_name=0)
data = pd.read_excel(r"D:\temp\Downloads\傲娇系二次元bot词库5千词V1.2.xlsx",header=None, sheet_name=0)
for index,row in data.iterrows():
    if row[0] in map:
        map[row[0]].append(row[1])
    else:
        map[row[0]] = [row[1]]
from service.MatchSys.trainer import QATrainer
trainer = QATrainer(ms)


In [ ]:
trainer.train(map)

In [ ]:
ms.get_response('石家庄天气怎么样')

In [1]:
from service import config
config.initialize()

from service.MatchSys import MatchSys
ms = MatchSys(
        name='teat_sys',
        ltp_model_path=r'C:\Users\Administrator\Documents\GitHub\QQ-Bot-And-Tool\data\LtpModel',
        database_uri='sqlite:///data/db.sqlite3',
        text_vec_model_path=r'C:\Users\Administrator\Documents\GitHub\QQ-Bot-And-Tool\data\model.pkl',
        vector_similarity_rate=0.7,
        most_similar_number=10,
        vector_match_times=5,

    )

@ms.handle_function_declaration(commands = ['天气怎么样'])
def get_weather_abstract_by_hefeng(input_statement):
    import requests
    from bs4 import BeautifulSoup
    get_location_url = 'https://geoapi.qweather.com/v2/city/lookup'
    address = input_statement.text.split('天气怎么样')[0]
    get_loction_params = {
        'key': config.SERVICE_CONFIG.he_feng_key,
        'location': address
    }
    location_data = requests.get(get_location_url,get_loction_params).json()
    print(location_data)
    location_data = location_data['location'][0]
    page = requests.get(location_data['fxLink'])
    soup = BeautifulSoup(page.text, 'html.parser')
    abstract = soup.find_all('div', class_='current-abstract')[0]
    return abstract.text.strip() + '\n 详细信息：'+location_data['fxLink']

1715766166338805760
Loading weights from local directory
# jionlp - 微信公众号: JioNLP  Github: `https://github.com/dongrixinyu/JioNLP`.
# jiojio - `http://www.jionlp.com/jionlp_online/cws_pos` is available for online trial.
Parrot started


In [2]:
ms.get_response('我爱你')

<class 'service.MatchSys.object_definition.Statement'>


: 